In [494]:
import pandas as pd
import numpy as np
from statsmodels.discrete.discrete_model import Probit
from statsmodels.iolib.summary2 import summary_col

## Simulation

In [518]:
# Simulate baseline survey data

def sim_baseline( N_participants = 5000,
                  
                  female_share = 0.6,
                  
                  race_shares = {
                      'black': 0.4,
                      'hispanic': 0.2,
                      'native_am': 0.05,
                      'white': 0.3
                      # the rest are assumed to have indicated no racial/ethnic group
                    },
                  
                  educ_shares = {
                      'middle_sch': 0.1, 
                      'high_sch': 0.4, 
                      'some_college': 0.1,
                      'college': 0.35
                      # the rest are assumed to have no education record
                    },
                  
                  labor_shares = {
                      'employed': 0.8,
                      'unemployed': 0.03
                      # the rest are assumed out of labor force
                  }
                  
                 ):
    
    # generate ids from 0 to 4999
    id = pd.Series(range(N_participants))

    # generate gender dummy randomly, according to female_share
    female = pd.Series(np.random.rand(5000) <= female_share).astype('int')
    
    # generate education dummies randomly, according to educ_shares
    educ_middle_sch = pd.Series([0] * 5000)
    educ_high_sch = pd.Series([0] * 5000)
    educ_some_college = pd.Series([0] * 5000)
    educ_college = pd.Series([0] * 5000)
    
    for i in range(5000):
        rnd = np.random.rand(1)[0]
        if rnd <= educ_shares['middle_sch']:
            educ_middle_sch[i] = 1
        elif rnd <= educ_shares['middle_sch'] + educ_shares['high_sch']:
            educ_high_sch[i] = 1
        elif rnd <= educ_shares['middle_sch'] + educ_shares['high_sch'] + educ_shares['some_college']:
            educ_some_college[i] = 1
        elif rnd <= educ_shares['middle_sch'] + educ_shares['high_sch'] + educ_shares['some_college'] + educ_shares['college']:
            educ_college[i] = 1
            
    # generate racial/ethnic dummies randomly, according to race_shares
    black = pd.Series([0] * 5000)
    hispanic = pd.Series([0] * 5000)
    native_am = pd.Series([0] * 5000)
    white = pd.Series([0] * 5000)
    
    for i in range(5000):
        rnd = np.random.rand(1)[0]
        if rnd <= race_shares['black']:
            black[i] = 1
        elif rnd <= race_shares['black'] + race_shares['hispanic']:
            hispanic[i] = 1
        elif rnd <= race_shares['black'] + race_shares['hispanic'] + race_shares['native_am']:
            native_am[i] = 1
        elif rnd <= race_shares['black'] + race_shares['hispanic'] + race_shares['native_am'] + race_shares['white']:
            white[i] = 1
            
    # generate age from log-normal distribution with mean increasing in education level
    age = np.exp(3.4 + educ_high_sch * 0.3 + educ_some_college * 0.35 + educ_college * 0.5 + 
                 np.random.normal(0, 0.2, 5000)).clip(18, 90).round(0)
    
    # generate employment dummies randomly, according to labor_shares
    employed = pd.Series([0] * 5000)
    unemployed = pd.Series([0] * 5000)
    
    for i in range(5000):
        rnd = np.random.rand(1)[0]
        if rnd <= labor_shares['employed']:
            employed[i] = 1
        elif rnd <= labor_shares['employed'] + labor_shares['unemployed']:
            unemployed[i] = 1
        
    # generate income from log-normal distribution, taking into account employment status and demographics
    income = pd.Series(np.exp(11 - 4 * unemployed + np.random.normal(0, 1, 5000))
                      + employed * \
                           (0.1 * educ_middle_sch + 0.12 * educ_high_sch + 0.2 * educ_some_college + 0.25 * educ_college \
                           -0.2 * black - 0.1 * hispanic - 0.3 * native_am + 0.1 * white \
                           + 0.01 * age - 0.2 * female + 1)
                      ).round(-3).clip(0)
    
    # generate vaccination dummy, taking into account demographics
    vaccinated_start = np.rint(np.random.normal(0.3, 0.3, 5000) \
                  + np.log(income + 1) / 100 \
                  + 0.15 * educ_some_college + 0.2 * educ_college \
                  - 0.01 * age + 0.0001 * age * age
                 ).astype('int').clip(0,1)
    
    # generate covid cases number, taking into account demographics
    covid_cases_start = np.exp(np.random.normal(0, 0.3, 5000) - 0.5 * vaccinated_start + 0.2 * employed).astype('int').clip(0)
    
    return pd.DataFrame({'id': id, 
                         'female': female, 
                         'educ_middle_sch': educ_middle_sch,
                         'educ_high_sch': educ_high_sch,
                         'educ_some_college': educ_some_college,
                         'educ_college': educ_college,
                         'black': black,
                         'hispanic': hispanic,
                         'native_am': native_am,
                         'white': white,
                         'age': age,
                         'employed': employed,
                         'unemployed': unemployed,
                         'income': income,
                         'vaccinated_start': vaccinated_start,
                         'covid_cases_start': covid_cases_start
                        })

In [519]:
baseline = sim_baseline()
baseline.to_csv('../data/raw/baseline_survey.csv')

In [520]:
def sim_assignment(N_participants = 5000):
    
    id = pd.Series(range(N_participants))
    
    rand_series = np.random.rand(5000)
    cutoff1 = np.quantile(rand_series, 1/3)
    cutoff2 = np.quantile(rand_series, 2/3)

    group = pd.Series([0] * 5000)
    group[rand_series < cutoff1] = 1
    group[rand_series > cutoff2] = 2
    
    # 0 - control, 1 - reason treatment, 2 - emotions treatment, 
    # group sizes: 1666, 1667, 1667
    
    return pd.DataFrame({'id': id, 
                         'group': group
                        })

In [521]:
assignment = sim_assignment()
assignment.to_csv('../data/raw/assignment.csv')

In [522]:
def sim_results(baseline = baseline,
               assignment = assignment):
    
    # "nature" sees the real data
    df = baseline.merge(assignment, on='id').copy()
    df['log_scaled_income'] = np.log(df['income'] + 1)
    df['log_scaled_income'] = (df['log_scaled_income'] - df['log_scaled_income'].mean()) / df['log_scaled_income'].std()
    
    # model attrition probability: age, employment, income, and being assigned to any treatment group increase it
    df['attrition_prob'] = np.random.normal(0.1, 0.05, 5000) + df['age'] * 0.0005 + df['employed'] * 0.01 \
        + np.log(df['income'] + 1) * 0.0001 - (df['group'] == 0).astype('int') * 0.05
    
    # find 500 individuals with highest probability of attrition and drop them
    attrition_ids = df['attrition_prob'].sort_values().tail(500).keys()
    df = df.drop(attrition_ids)
    
    # model vaccination during program: higher rates among those who had covid before, several racial & gender group;
    # intervention 1 has no effect,
    # intervention 2 has positive effect on the rich and the young, while negative on the poor and the old
    vaccination = (
        np.random.normal(0.2, 0.2, 4500) + df['covid_cases_start'] * 0.05 \
        + df['black'] * 0.01 + df['hispanic'] * 0.02 + df['female'] * 0.03 \
        + (df['group'] == 2).astype('int') * (df['log_scaled_income'] / 5 + (40 - df['age']) / 40)
    )
        
    df['vaccination'] = np.rint(vaccination).astype('int')
    df['vaccinated_end'] = (df['vaccinated_start'] + df['vaccination']).clip(0,1)

    # save only the observable outcome: whether an individual is vaccinated by endline survey
    return df[['id', 'vaccinated_end']]

In [523]:
endline = sim_results()
endline.to_csv('../data/raw/endline_survey.csv')

## Analysis

In [524]:
# Load the data
baseline = pd.read_csv('../data/raw/baseline_survey.csv', index_col=0)
assignment = pd.read_csv('../data/raw/assignment.csv', index_col=0)
endline = pd.read_csv('../data/raw/endline_survey.csv', index_col=0)


# Merge
baseline_and_group = baseline.merge(assignment, on='id', how='inner')
df = baseline_and_group.merge(endline, on='id', how='left')

# Display and save the merged data
display(df.sample(5))
df.to_csv('../data/experiment_merged.csv')

,id,female,educ_middle_sch,educ_high_sch,educ_some_college,educ_college,black,hispanic,native_am,white,age,employed,unemployed,income,vaccinated_start,covid_cases_start,group,vaccinated_end
1195,1195,1,0,0,1,0,0,1,0,0,36.0,0,1,0.0,0,0,2,0.0
383,383,1,0,0,0,1,0,1,0,0,42.0,1,0,422000.0,0,1,2,1.0
4184,4184,0,1,0,0,0,0,0,0,1,24.0,1,0,66000.0,0,1,0,0.0
157,157,1,0,1,0,0,0,0,0,1,30.0,1,0,67000.0,1,0,2,NaN
4220,4220,1,0,0,0,1,0,0,0,0,61.0,1,0,191000.0,1,0,0,1.0


In [525]:
# Drop N/A
# ! this step in the analysis introduces bias into the regression without controls due to non-random attrition.

df = df.dropna()

# Drop the initially vaccinated since the effect in this group is not observable
df = df[df['vaccinated_start'] == 0]

# Generate treatment dummies
df['treatment_reason'] = (df['group'] == 1).astype('int')
df['treatment_emotion'] = (df['group'] == 2).astype('int')

# Add intercept
df['intercept'] = 1

# Generate log of income, add 1 to account for zeros
df['log_income'] = np.log(df['income'] + 1)

# Generate age²
df['age2'] = df['age'] * df['age']

In [526]:
# List all regressors for a specification with controls
model_controls_regressors = ['intercept', 'treatment_reason', 'treatment_emotion', 'covid_cases_start',
               'female', 'educ_middle_sch', 
               'educ_high_sch', 'educ_some_college', 'educ_college', 'black', 'hispanic', 'native_am', 'white', 
               'age', 'age2', 'employed', 'unemployed', 'log_income']

# Simplest model (biased due to attrition)
model_simple = Probit(
    exog = df[['intercept', 'treatment_reason', 'treatment_emotion']],
    endog = df['vaccinated_end']
)

# Model with controls 
model_controls = Probit(
    exog = df[model_controls_regressors],
    endog = df['vaccinated_end']
)

In [533]:
# Create & save the output regression table
models_full_sample = summary_col([model_no_controls.fit(), model_controls.fit()],
                       stars=True,
                       regressor_order=model_controls_regressors)

print(models_full_sample)

f = open("../results/full_sample.tex", "w")
f.write(models_full_sample.as_latex())
f.close()

Optimization terminated successfully.
         Current function value: 0.424526
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.394422
         Iterations 6

                  vaccinated_end I vaccinated_end II
----------------------------------------------------
intercept         -1.1336***       -1.0429**        
                  (0.0451)         (0.4699)         
treatment_reason  0.0643           0.0019           
                  (0.0642)         (0.0671)         
treatment_emotion 0.2544***        0.3658***        
                  (0.0624)         (0.0646)         
covid_cases_start                  0.2236***        
                                   (0.0535)         
female                             0.1634***        
                                   (0.0557)         
educ_middle_sch                    0.1677           
                                   (0.1293)         
educ_high_sch                      0.1364           


In [528]:
# Drop the inefficient "reason" treatment
df_emotion = df[df['treatment_reason'] != 1].copy()

# Generate treatment_emotion X var interactions

control_vars = ['covid_cases_start', 'female', 'age', 'age2', 'log_income']
treatment_x_cv = [f'treatment_x_{cv}' for cv in control_vars]

for cv in control_vars:
    df_emotion[f'treatment_x_{cv}'] = df['treatment_emotion'] * df[cv]

# Model with controls + effect heterogeneity captured by treatment X var interactions
model_heterog = Probit(
    exog = df_emotion[['intercept', 'treatment_emotion', *control_vars, *treatment_x_cv]],
    endog = df_emotion['vaccinated_end']
)

In [529]:
model_heterog.fit().summary()

Optimization terminated successfully.
         Current function value: 0.339841
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:         vaccinated_end   No. Observations:                 2348
Model:                         Probit   Df Residuals:                     2336
Method:                           MLE   Df Model:                           11
Date:                Tue, 19 Dec 2023   Pseudo R-squ.:                  0.2395
Time:                        05:16:51   Log-Likelihood:                -797.95
converged:                       True   LL-Null:                       -1049.3
Covariance Type:            nonrobust   LLR p-value:                8.577e-101
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
intercept                        -0.7624      0.638     -1.194      0.232      -2.014       0.489
treatment_emotion                -3.6714      1.463     -2.510      0.012      -6.538      -0.804
covid_cases_start                 0.2707      0.090      3.024      0.002       0.095       0.446
female                            0.1689      0.096      1.762      0.078      -0.019       0.357
age                              -0.0110      0.026     -0.424      0.672      -0.062       0.040
age2                           6.954e-05      0.000      0.238      0.812      -0.001       0.001
log_income                       -0.0365      0.030     -1.230      0.219      -0.095       0.022
treatment_x_covid_cases_start     0.0991      0.140      0.707      0.479      -0.176       0.374
treatment_x_female                0.1389      0.149      0.934      0.350      -0.152       0.430
treatment_x_age                  -0.0678      0.070     -0.963      0.335      -0.206       0.070
treatment_x_age2                 -0.0006      0.001     -0.662      0.508      -0.002       0.001
treatment_x_log_income            0.6688      0.068      9.894      0.000       0.536       0.801
=================================================================================================
"""

In [532]:
# Save the result
model_emotion = summary_col([model_heterog.fit()],
                       stars=True,
                       regressor_order=['intercept', 'treatment_emotion', *control_vars, *treatment_x_cv])


f = open("../results/treatment_emotion.tex", "w")
f.write(model_emotion.as_latex())
f.close()

Optimization terminated successfully.
         Current function value: 0.339841
         Iterations 9
